In [105]:
import sys; sys.path.append('..')
from osp import *

In [178]:
for id,docstr in STASH_SLICES_NLP.items():
    break
doc=stanza.Document.from_serialized(docstr)
sent = random.choice(doc.sentences)
sent.text

'However, the fundamental objection holds against this conception of Abstract General Ideas as well as against the other, as Locke himself points out.'

In [179]:
def is_leaf(tree):
    """Check if a tree node is a leaf (preterminal) node."""
    return not isinstance(tree[0], nltk.Tree) if len(tree) > 0 else False


def get_leaf_ids(tree):
    print(tree)
    return [word.id for word in tree.leaves()]

def iter_preterms(tree):
    for t in tree.subtrees():
        if is_leaf(t):
            yield t

def tokenize_clauses(tree): # -> return both whether "IC" or "DC"
    clause_num = 0
    clause_type = None
    for t in tree.subtrees():
        ttxt = ' '.join(t.leaves())
        if ttxt and ttxt[0].isalpha() and ((not is_in_sbar(t) and t.label() == 'S') or t.label() == 'SBAR'):
            clause_num += 1
            clause_type = 'DC' if is_in_sbar(t) else 'IC'

        t._clause_num = clause_num
        t._clause_type = clause_type

    # o = defaultdict(list)
    o = []
    for preterm_i,preterm in enumerate(iter_preterms(tree)):
        d = {
            'clause_num': preterm._clause_num,
            'clause_type': preterm._clause_type,
            'word_num': preterm_i+1,
            'word': ' '.join(preterm.leaves())
        }
        o.append(d)
    
    odf = pd.DataFrame(o)
    odf['clause_num'] = odf['clause_num'].rank(method='dense', ascending=True).apply(int)
    return odf
    

In [180]:
# sent = get_nlp_doc("The trouble about progress is that it always looks much greater than it really is.").sentences[0]
# # sent

In [181]:
def get_clauses(sent):
    if isinstance(sent, str):
        sent = get_nlp_doc(sent).sentences[0]
    tree = get_sent_tree(sent)
    odf = tokenize_clauses(tree)
    assert len(odf) == len(sent.words)
    odf['pos'] = [w.xpos for w in sent.words]
    odf['deprel'] = [w.deprel for w in sent.words]
    odf['head'] = [w.head for w in sent.words]
    return odf.set_index(['clause_num','clause_type','word_num'])

get_clauses(sent)

word  pos         deprel  head
clause_num clause_type word_num                                       
1          IC          1             However   RB         advmod     6
                       2                   ,    ,          punct     1
                       3                 the   DT            det     5
                       4         fundamental   JJ           amod     5
                       5           objection   NN          nsubj     6
                       6               holds  VBZ           root     0
                       7             against   IN           case     9
                       8                this   DT            det     9
                       9          conception   NN            obl     6
                       10                 of   IN           case    13
                       11           Abstract   JJ           amod    13
                       12            General   JJ           amod    13
                       13              Ideas  NNS           nmod     9
                       14                 as   RB             cc    19
                       15               well   RB          fixed    14
                       16                 as   IN          fixed    14
                       17            against   IN           case    19
                       18                the   DT            det    19
                       19              other   JJ           conj    13
                       20                  ,    ,          punct    24
2          IC          21                 as   IN           mark    24
                       22              Locke  NNP          nsubj    24
                       23            himself  PRP  nmod:unmarked    22
                       24             points  VBZ          advcl     6
                       25                out   RP   compound:prt    24
                       26                  .    .          punct     6

In [182]:
# sent.words[0]

In [183]:
df = get_clauses(sent)
for clause, clause_df in df.groupby('clause_num'):
    break

In [184]:
@cache
def get_df_pred_feats():
    df_feats = pd.read_excel('../data/feats_perspectival2.xlsx').drop(columns=['Unnamed: 0']).groupby('feature').mean(numeric_only=True)
    return df_feats
df_feats = get_df_pred_feats()
df_feats

,weight,run,score_mean1,score_mean2,score_z1,score_z2,score_mean_diff,score_mean_diff_abs,score_mean_diff_pct,score_mean_div,score_mean_div_abs,score_z_diff,score_z_diff_abs,score_z_diff_pct,score_z_div,score_z_div_abs
feature,,,,,,,,,,,,,,,,
deprel_acl,0.142384,0.0,11.216790,9.107505,0.118450,-0.375904,2.109285,2.109285,0.230526,1.230526,1.230526,0.494354,0.494354,-1.550019,-0.550019,0.708506
deprel_acl:relcl,-0.181810,0.0,14.039901,12.588054,0.085020,-0.217207,1.451847,1.451847,0.125239,1.125239,1.125239,0.302227,0.302227,46.651940,47.651940,50.927767
deprel_advcl,-0.143057,0.0,19.189076,16.558620,0.114072,-0.353766,2.630456,2.630456,0.158108,1.158108,1.158108,0.467838,0.467838,-1.495189,-0.495189,0.669301
deprel_advcl:relcl,0.088687,0.0,0.613740,0.421110,0.017397,-0.209981,0.192630,0.192630,0.433010,1.433010,1.433010,0.227378,0.227378,-1.720903,-0.720903,1.283101
deprel_advmod,0.263665,0.0,54.854743,48.439715,0.148969,-0.381908,6.415028,6.415028,0.132838,1.132838,1.132838,0.530878,0.530878,-1.396873,-0.396873,0.418074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sent_height,0.653442,0.0,503.589011,460.239806,0.193973,-0.522419,43.349205,43.349205,0.094690,1.094690,1.094690,0.716392,0.716392,-1.305544,-0.305544,0.430568
sent_num_dependent_clauses,0.652998,0.0,46.230107,34.289937,0.261591,-0.670961,11.940171,11.940171,0.347351,1.347351,1.347351,0.932552,0.932552,-1.415609,-0.415609,0.467027
sent_num_independent_clauses,0.088265,0.0,58.607335,58.280006,0.020305,-0.007389,0.327330,2.155116,0.006914,1.006914,1.006914,0.027694,0.185823,-1.134986,-0.134986,0.346939


In [185]:
# !pip install colour

In [249]:
from colour import Color
color_range = list(Color('blue').range_to(Color('gray'), 50)) + list(Color('gray').range_to(Color('red'), 50))
# list(color_range)

In [250]:
df_feats.loc['deprel_mark']

weight                  1.112859
run                     0.000000
score_mean1            45.845501
score_mean2            32.560528
score_z1                0.267694
score_z2               -0.679417
score_mean_diff        13.284973
score_mean_diff_abs    13.284973
score_mean_diff_pct     0.406794
score_mean_div          1.406794
score_mean_div_abs      1.406794
score_z_diff            0.947111
score_z_diff_abs        0.947111
score_z_diff_pct       -1.414195
score_z_div            -0.414195
score_z_div_abs         0.509241
Name: deprel_mark, dtype: float64

In [251]:
def get_color(
    feat,
    color_by='weight',
    df_feats=None,
):
    if df_feats is None:
        df_feats = get_df_pred_feats()
    featval = df_feats.loc[feat, color_by] if feat in df_feats.index else 0
    # color_by_min = float(df_feats[color_by].fillna(0).min())
    # color_by_max = float(df_feats[color_by].fillna(0).max())
    color_by_min = -1
    color_by_max = 1
    color_by_range = color_by_max - color_by_min
    color_idx = int(100 * (featval - color_by_min) / color_by_range)
    try:
        return color_range[color_idx].hex
    except:
        return '#000000'

In [252]:
df_feats['weight'].min()

np.float64(-0.9565587453551221)

In [253]:
get_color('pos_VB')

'#be4141'

In [259]:
def get_manual_displacy_data(clause_df):
    clause_data = {
        "words": [{"text": text, "tag": pos} for text,pos in zip(clause_df['word'], clause_df['pos'])],
        "arcs": [
            {
                "start": row['word_num']-1,
                "end": row['head']-1,
                "label": row['deprel'],
                "dir": "left" if row['word_num'] > row['head'] else "right"
            }
            for _, row in clause_df.reset_index().iterrows()
            if row['word_num'] < row['head']
        ]
    }
    return clause_data

In [260]:
clause_data = get_manual_displacy_data(df)

In [261]:
# for clause, clause_df in df.groupby('clause_num'):
#     clause_data = get_manual_displacy_data(clause_df)
#     displacy.render(clause_data, style="dep", manual=True)
#     # x.render_to_file(f"clause_{clause}.svg")

In [262]:
# !pip install beautifulsoup4
from bs4 import BeautifulSoup

In [263]:
from IPython.display import HTML, display
from spacy import displacy
html_output = displacy.render(clause_data, style="dep", manual=True,options={'compact': True},jupyter=False)

# Parse the SVG
soup = BeautifulSoup(html_output, 'html.parser')

tag_elements = soup.find_all('tspan', class_='displacy-tag')
for idx, tag_elem in enumerate(tag_elements):
    tag_elem['fill'] = get_color(f'pos_{tag_elem.text}', color_by='score_z_diff')

deprel_elements = soup.find_all('textpath')
for idx, deprel_elem in enumerate(deprel_elements):
    deprel_elem['fill'] = get_color(f'deprel_{deprel_elem.text}', color_by='score_z_diff')

# # Combine and display
HTML(str(soup))
# # deprel_elements

In [264]:
# print(html_output )

In [17]:
stopxx

NameError: name 'stopxx' is not defined

In [16]:
displacy.serve(clause_data, style="dep", manual=True, port=5001)


Using the 'dep' visualizer
Serving on http://0.0.0.0:5001 ...



127.0.0.1 - - [25/Dec/2025 00:29:25] "GET / HTTP/1.1" 200 11108
127.0.0.1 - - [25/Dec/2025 00:29:25] "GET /favicon.ico HTTP/1.1" 200 11108


Shutting down server on port 5001.


In [17]:
# Format your Stanza data into this specific dictionary structure
manual_data = {
    "words": [
        {"text": "Barack", "tag": "PROPN"},
        {"text": "Obama", "tag": "PROPN"},
        {"text": "was", "tag": "AUX"},
        {"text": "born", "tag": "VERB"}
    ],
    "arcs": [
        {"start": 0, "end": 1, "label": "flat", "dir": "left"},
        {"start": 1, "end": 3, "label": "nsubjpass", "dir": "left"},
        {"start": 2, "end": 3, "label": "auxpass", "dir": "left"}
    ]
}

displacy.render(manual_data, style="dep", manual=True)

In [ ]:
claus

In [26]:
def get_displacy_data_from_sent(sent):
    words = [{"text": word.text, "tag": word.xpos} for word in sent.words]
    arcs = []
    
    for word in sent.words:
        # Skip root dependencies (head == 0)
        if word.head == 0:
            continue
        
        # Convert from 1-based (Stanza) to 0-based (displaCy) indexing
        start = word.head - 1  # head position (0-based)
        end = word.id - 1      # current word position (0-based)
        
        # Skip self-loops (word pointing to itself)
        if start == end:
            continue
        
        # Determine direction
        if start < end:
            direction = "left"
        else:
            direction = "right"
            # Swap start and end for right-pointing arcs
            start, end = end, start
        
        arcs.append({
            "start": start,
            "end": end,
            "label": word.deprel,
            "dir": direction
        })
    
    return {"words": words, "arcs": arcs}

data = get_displacy_data_from_sent(sent)

In [27]:
displacy.render(data, style="dep", manual=True)